### Overview of this notebook

* overview of `sktime` as a framework - estimators modules, library, data
* overview of learning tasks in `sktime` - API perspective
* searching the library for estimators, tag system
* estimator level dependency management
* creating your own estimator, to `sktime`, or for third party use (closed or open) - short primer

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 5. `sktime` in a nutshell - engineering perspective

**A) `sktime` is a modular framework for multiple learning tasks**

Example: forecasting (predict future of ts), classification (predict label of ts)

**B) estimators/algorithms are of a scientific type = which task do they solve?**

Example: ARIMA is a forecaster; knn with time series distance is classifier

**C) all estimators of a certain scitype have the same module interface**

Example: all forecasters classes have `fit` / `predict` with same contract

**D) `sktime` is a library which allows browsing of integrated estimators**

Example: search for all forecasters that are natively multivariate

**E) `sktime` is a mini-package manager for estimators and their dependencies**

Example: `ARIMA` class requires `pmdarima`, but `sktime` itself does not

**F) `sktime` is extensible, write your own 3rd party plugins (open or closed)**

Example: forecaster in 3rd party codebase, plug & plays to `sktime` and test framework

**G) `sktime` is highly composable - any compatible estimators of any type!**

Example: pipeline 3rd party feature extractor and onboard detrender with forecaster

## 5.1 Showcase with code vignettes

The above, with code. We revisit in more detail later.

### **A) `sktime` is a modular framework for multiple learning tasks**

Vignettes for forecasting and classification:
(we'll go into data types etc later)

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.naive import NaiveForecaster
import numpy as np

# step 1: data specification
y = load_airline()
# y is a pd.Series at monthly frequency

# step 2: specifying forecasting horizon
fh = np.arange(1, 37)
# this specifies a prediction 3 years ahead

# step 3: specifying the forecasting algorithm
forecaster = NaiveForecaster(strategy="last", sp=12)
# forecaster is now a forecaster object of type NaiveForecaster

# step 4: fitting the forecaster
forecaster.fit(y, fh=fh)
# forecaster changes state to "fitted"

# step 5: querying predictions
y_pred = forecaster.predict()
# y_pred is the forecasted time series, a pd.Series

In [ ]:
from sktime.datasets import load_osuleaf
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

# step 1 - specify training data
X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
# X_train is 3D numpy array holding multiple instances of time series
# y_train is 1D numpy array with training labels for these instances

# step 2 - specify data to predict labels for
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]
# X_new is a 3D numpy array with the instances to label

# step 3 - specify the classifier
mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)
# clf is a classifier object of type KNeighborsTimeSeriesClassifier
# it consists of other sktime objects, mean_eucl_dist is a distance object

# step 4 - fitting the classifier
clf.fit(X_train, y_train)
# clf changes state to "fitted"

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)
# y_pred is the predicted labels, an 1D numpy array

### **B) estimators/algorithms are of a scientific type = which task do they solve?**

`NaiveForecaster` is a forecaster; `KNeighborsTimeSeriesClassifier` is a classifier

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.registry import scitype

scitype(NaiveForecaster)

In [ ]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.registry import scitype

scitype(KNeighborsTimeSeriesClassifier)

**C) all estimators of a certain scitype have the same module interface**

C1 - the `NaiveForecaster` can be switched out for any forecaster in the base vignette

Only step 3 - specification changes!

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA
import numpy as np

# step 1: data specification
y = load_airline()

# step 2: specifying forecasting horizon
fh = np.arange(1, 37)

# step 3: specifying the forecasting algorithm
# forecaster = NaiveForecaster(strategy="last", sp=12)
forecaster = ARIMA()

# step 4: fitting the forecaster
forecaster.fit(y)

# step 5: querying predictions
y_pred = forecaster.predict(fh)

C2 - `fit` and `predict` are the same for both!

`fit(self, y, X=None, fh=None)`

In [ ]:
?ARIMA.fit

In [ ]:
?NaiveForecaster.fit

for classifiers, signature is different: `fit(X, y)`

but the same for all classifiers!

In [ ]:
KNeighborsTimeSeriesClassifier.fit

### **D) `sktime` is a library which allows browsing of integrated estimators**

Example: search for all forecasters that can make probabilistic predictions

In [ ]:
from sktime.registry import all_estimators

all_estimators("forecaster", filter_tags={"capability:pred_int": True}, as_dataframe=True)

all objects in `sktime` are tagged with metadata:

In [ ]:
ARIMA().get_tags()

list all tags that apply to forecasters:

In [ ]:
from sktime.registry import all_tags

all_tags("forecaster", as_dataframe=True)

### **E) `sktime` is a mini-package manager for estimators and their dependencies**

Example: `ARIMA` class requires `pmdarima`, but `sktime` itself does not

In [ ]:
from sktime.forecasting.arima import ARIMA

ARIMA.get_class_tag("python_dependencies")
# this requires the pmdarima package
# the result is a PEP 440 compatible requirement string

by default, dependencies are checked at instantiation:

In [ ]:
# from sktime.forecasting.fbprophet import Prophet

# Prophet()

# this would result in:

this would result in exception:

```
ModuleNotFoundError: Prophet requires package 'prophet' to be present in the python environment,
but 'prophet' was not found. 'prophet' is a soft dependency and not included in the base
sktime installation. Please run: `pip install prophet` to install the prophet package.
To install all soft dependencies, run: `pip install sktime[all_extras]`
```

### **F) `sktime` is extensible, write your own 3rd party plugins (open or closed)**

Example: forecaster in 3rd party codebase, plug & plays to `sktime` and test framework

snippet from forecaster extension template (in `extension_templates` dir):
```
How to use this implementation template to implement a new estimator:
- make a copy of the template in a suitable location, give it a descriptive name.
- work through all the "todo" comments below
- fill in code for mandatory methods, and optionally for optional methods
- do not write to reserved variables: is_fitted, _is_fitted, _X, _y, cutoff, _fh,
    _cutoff, _converter_store_y, forecasters_, _tags, _tags_dynamic, _is_vectorized
- you can add more private methods, but do not override BaseEstimator's private methods
    an easy way to be safe is to prefix your methods with "_custom"
- change docstrings for functions and the file
- ensure interface compatibility by sktime.utils.estimator_checks.check_estimator
- once complete: use as a local library, or contribute to sktime via PR
- more details:
  https://www.sktime.net/en/stable/developer_guide/add_estimators.html
```

![](./img/implementing_estimators.png)

In [ ]:
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.utils.estimator_checks import check_estimator

res = check_estimator(BoxCoxTransformer)

In [ ]:
res

**G) `sktime` is highly composable - any compatible estimators of any type!**

Example: pipeline 3rd party feature extractor and onboard detrender with forecaster

In [ ]:
from sklearn.linear_model import LinearRegression

from sktime.forecasting.arima import ARIMA
from sktime.forecasting.trend import TrendForecaster
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.series.detrend import Detrender

trend_reg = LinearRegression()
detrender = Detrender(TrendForecaster(trend_reg))

pipe = BoxCoxTransformer() * detrender * ARIMA()

# we first box-cox-transform - from statsmodels
# then we detrend - sktime native
# detrending is done using a regression forecaster - sktime native
# the regression in the trend forecaster is linear regressor - from sklearn
# then we apply ARIMA - from pmdarima

# when forecasting, the detrending and box cox transform are inverted, in this order

# this is how it looks like - highly modular!
# any step could be switched out and can be tuned with grid search etc
pipe

In [ ]:
# tunable parameters
pipe.get_params()

## 5.2 Learning tasks in sktime

Step 1 - what is your learning task?

sktime estimator type support:

| Task | Status | Links |
|---|---|---|
| **Forecasting** | stable | [Tutorial](https://www.sktime.net/en/latest/examples/01_forecasting.html) · [API Reference](https://www.sktime.net/en/latest/api_reference/forecasting.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/forecasting.py)  |
| **Time Series Classification** | stable | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/02_classification.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/classification.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/classification.py) |
| **Time Series Regression** | stable | [API Reference](https://www.sktime.net/en/latest/api_reference/regression.html) |
| **Transformations** | stable | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/transformations.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)  |
| **Parameter fitting** | maturing | [API Reference](https://www.sktime.net/en/latest/api_reference/param_est.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)  |
| **Time Series Clustering** | maturing | [API Reference](https://www.sktime.net/en/latest/api_reference/clustering.html) ·  [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/clustering.py) |
| **Time Series Distances/Kernels** | maturing | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/dists_kernels.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/dist_kern_panel.py) |
| **Annotation** | experimental | [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/annotation.py) |
| **Distributions and simulation** | experimental |  |

rough overview of time series related learning tasks

![](./img/ts-tasks.jpg)

first some basic terminology on time series (required for the above)

### 5.2.1 learning task guide - primer

questions to ask:

1. what is my data? one time series, panel of time series?
2. what do I want to do with it: predict? transform? annotate?
3. what data comes out of the model when it is applied?
4. how do I train? on what data? (or do I not train?)

starting with explaining common tasks:

* forecasting
* simple panel data tasks - classification, regression, clustering
* panel forecasting

#### **forecasting** (vanilla)

![](./img/tasks-forecasting.png)

e.g., **given this**:

In [ ]:
from sktime.datasets import load_macroeconomic

y = load_macroeconomic()
y

**algorithm, please fill in rows for 2009Q4, 2010Q1, ...**

if want only some columns (variables) for 2009Q4:

* columns to forecast are called "endogenous"
* other columns, if used by algorithm, are called "exogenous"
* some exogenous columns may be known for future (e.g., interest rates), some may be unknown

#### **time series classification** and regression

![](./img/tasks-tsc.png)

e.g., **given this**:

In [ ]:
from sktime.datasets import load_basic_motions

X, _ = load_basic_motions(split="TEST", return_type="pd-multiindex")
X

**algorithm, please assign a label to instance 0, a label to instance 1, ..., to instance 39**

label is a category (e.g., unifoliate leaf, bifoliate leaf, ...) -> time series classification

label is a number (e.g., 1, 3.14, 42) -> time series regression

**you can train on some instances with known label**:

In [ ]:
from sktime.datasets import load_basic_motions

X_train, y_train = load_basic_motions(split="TRAIN", return_type="pd-multiindex")
X_train, y_train

#### **time series clustering**

data as above, but there is no labelling or training data

algorithm should do one or more of of:

* put the series into a small number of buckets (prototype 1, prototype 2, ...)
* compute a similarity hierarchy between the instances
* produce a parsimonious clustering model summary (algorithm dependent)

#### **panel forecasting**

= for a panel of time series, forecast all (or only some) of the instances

e.g., **given this**:

In [ ]:
from sktime.utils._testing.hierarchical import _make_hierarchical

y = _make_hierarchical(hierarchy_levels=(4,), min_timepoints=4, max_timepoints=5)
y

**algorithm, please forecast 2000-01-06, 2000-01-07 for instances h0_0, h0_1, ...**

optional: full training instances are available that are not being forecast (sometimes called"global" forecasting)

IMPORTANT: not the same as time series regression! Not the same as tabular regression!

Forecasts are numbers, but they are indexed by time - regression label is not time indexed.

#### further and rarer learning tasks

for disambiguation - experimental or no support in `sktime` (contribute!)

* prediction is also time series
    * concurrent on same time axis: supervised annotation, nowcasting
    * future, same time axis: global or supervised forecasting
    * unrelated axis: functional regression
* prediction is summary or transform of time series
    * directional forecasting
    * annotation forecast
* detecting anomalies, changepoints
    * temporal anomaly detection, changepoint detection
    * latent space modelling, unsupervised annotation

most above tasks can be:

* panel/hierarchical or not
* supervised or not, training data varying across index
* batch or on-line
* combined with output modality: point prediction, probabilistic, event

`sktime` roadmap to cover "task space" in order of commonality

contributions appreciated!

### 5.2.1 Time Series Forecasting

Basic deployment vignette for forecasting:

1. load/setup training data, `y` as `pd.DataFrame`
2. specify the forecasting horizon, i.e., what to forecast
3. specify the forecaster using `sklearn`-like syntax
4. fit forecaster to training data, `fit(y, fh)`
5. compute forecasts, `predict()`

In [ ]:
# step 1 - prepare macroeconomic data
from sktime.datasets import load_macroeconomic

y = load_macroeconomic().iloc[:-1]  # we leave out the last row for later

In [ ]:
y

In [ ]:
# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

In [ ]:
# step 3 - specify the forecasting algorithm
from sktime.forecasting.naive import NaiveForecaster

# this predicts the last value seen
forecaster = NaiveForecaster(strategy="last")

we could specify any `sktime` forecaster here - the rest remains the same!

In [ ]:
# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

In [ ]:
# the forecaster is now fitted
forecaster.is_fitted

In [ ]:
# and we can inspect fitted parameters if we like
forecaster.get_fitted_params()

In [ ]:
# step 5 - compute forecasts
y_pred = forecaster.predict()

In [ ]:
y_pred

all together in one cell:

In [ ]:
from sktime.datasets import load_macroeconomic

from sktime.forecasting.naive import NaiveForecaster

# step 1 - prepare macroeconomic data
y = load_macroeconomic()

# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

# step 3 - specify the forecasting algorithm
forecaster = NaiveForecaster(strategy="last")

# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

# step 5 - compute forecasts
y_pred = forecaster.predict()

sktime allows online/stream mode with forecaster update:

suppose tine now passes and we get new data in 2009Q3

In [ ]:
y_new = load_macroeconomic().iloc[-1:]  # the row we left out, let's pretend it's new

In [ ]:
y_new

we can update the forecaster with it:

In [ ]:
forecaster.update(y_new)

and we can get new forecasts, again for one year ahead (the forecaster remembers the `fh`):

In [ ]:
forecaster.predict()

### 5.2.2 Panel forecasting

panel forecasting does not have a separate estimator type. All forecasters can do this.

Passing panel data is enough for this:

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.utils._testing.hierarchical import _make_hierarchical

# step 1 - prepare some panel data
y = _make_hierarchical(hierarchy_levels=(4,), min_timepoints=4, max_timepoints=4)

# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

# step 3 - specify the forecasting algorithm
forecaster = NaiveForecaster(strategy="last")

# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

# step 5 - compute forecasts
y_pred = forecaster.predict()

In [ ]:
y.head()

In [ ]:
y_pred.head()

### 5.2.3 Time Series Classification

Basic deployment vignette for TSC:

1. load/setup training data, `X` in a `Panel` format, `y` as 1D `np.ndarray`
2. load/setup new data for prediction (can be done after 2 too)
3. specify the classifier using `sklearn`-like syntax
4. fit classifier to training data, `fit(X, y)`
5. predict labels on new data, `predict(X_new)`

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_osuleaf

X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]  # smaller dataset for faster notebook runtime

In [ ]:
# this is in numpy3D format, but could also be pd-multiindex or other
X_train.shape

In [ ]:
# y is a 1D np.ndarray of labels - same length as number of instances in X_train
y_train.shape

In [ ]:
# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3)

# example 2:
# 3-nearest neighbour classifier with mean (over time points) pairwise Euclidean distance
# (requires scipy)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)

we could specify any `sktime` classifier here - the rest remains the same!

In [ ]:
# all classifiers is scikit-learn / scikit-base compatible!
# nested parameter interface via get_params, set_params
clf.get_params()

In [ ]:
# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

In [ ]:
# the classifier is now fitted
clf.is_fitted

In [ ]:
# and we can inspect fitted parameters if we like
clf.get_fitted_params()

In [ ]:
# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [ ]:
# y_pred is an 1D np.ndarray, similar to sklearn classification output
y_pred

all together in one cell:

In [ ]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_osuleaf

X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]  # smaller dataset for faster notebook runtime

# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)

# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

evaluation same as in sklearn - use `accuracy` etc

### 5.2.4 Time Series Regression

TSR vignettes are exactly the same as TSC, except that:

* `y` in `fit` input and `predict` output should be float 1D `np.ndarray`, not categorical
* other algorithms are commonly used and/or performant

In [ ]:
# steps 1, 2 - prepare dataset (train and new)
from sktime.datasets import load_covid_3month

X_train, y_train = load_covid_3month(split="train")
y_train = y_train.astype("float")
X_new, _ = load_covid_3month(split="test")
X_new = X_new.loc[:2]  # smaller dataset for faster notebook runtime

# step 3 - specify the regressor
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

clf = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance=mean_eucl_dist)

# step 4 - fit/train the regressor
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [ ]:
y_pred  # not too interesting but float

### 5.2.5 Time Series Clustering

TS clustering is similar - 1st step is also `fit`, but unsupervised

i.e., no labels `y`, and next step is inspecting clusters

In [ ]:
from sktime.clustering.dbscan import TimeSeriesDBSCAN

# step 1 - prepare dataset (train and new)
X, _ = load_osuleaf(split="train", return_type="numpy3D")
X = X[:10]

# step 2 - specify the clusterer
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clst = TimeSeriesDBSCAN(distance=mean_eucl_dist)

# step 3 - fit the clusterer to the data
clst.fit(X)

# step 4 - inspect the clustering
clst.get_fitted_params()

## 5.3 Searching for estimators, estimator tags

Estimators in `sktime` are tagged.

User facing tags indicate things the estimator can or cannot do, these usually
start with "capability" (but not all, for historical reasons).

Examples:

forecasters

* `"capability:pred_int"` - can make probabilistic predictions (`predict_interval` etc)
* `"ignores-exogeneous-X"` - can make use of exogeneous data or not
* `"handles-missing-data"` - dealing with missing values

classifiers

* `"capability:missing_values"` - dealing with missing values
* `"capability:multivariate"` - daeling with multivariate input
* `"capability:unequal_length"` - dealing with time series panels where the individual time series have unequal length and/or unequal index

all tags for an estimator scitype (e.g., classifier, regressor) can be inspected by `sktime.registry.all_tags`:

In [ ]:
from sktime.registry import all_tags

all_tags("classifier", as_dataframe=True)

valid estimator types are listed in the `all_tags` docstring, or `sktime.registry.BASE_CLASS_REGISTER`

In [ ]:
from sktime.registry import BASE_CLASS_REGISTER

# get only fist table column, the list of types
list(zip(*BASE_CLASS_REGISTER))[0]

to find all estimators of a certain type, use `sktime.registry.all_estimators`

In [ ]:
# list all forecasters in sktime
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True)

for listing all estimators of a certain type with a certain capability,
use the `filter_tags` argument of `all_estimators`:

In [ ]:
# list all forecasters in sktime
# that can make probabilistic forecasts, e.g., prediction interval forecasts
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True, filter_tags={"capability:pred_int": True})

## 5.4 Custom estimators - extension guide

![](./img/implementing_estimators.png)

`sktime` is meant to be easily extensible, for direct contribution to `sktime` as well as for local/private extension with custom methods.

To extend `sktime` with a new local or contributed estimator, a good workflow to follow is:

0. find the right extension template for the type of estimator you want to add - e.g., classifier, regressor, clusterer, etc. The extension templates are located in the [`extension_templates`](https://github.com/sktime/sktime/blob/main/extension_templates) directory
1. read through the extension template - this is a `python` file with `todo` blocks that mark the places in which changes need to be added.
2. optionally, if you are planning any major surgeries to the interface: look at the base class - note that "ordinary" extension (e.g., new algorithm) should be easily doable without this.
3. copy the extension template to a local folder in your own repository (local/private extension), or to a suitable location in your clone of the `sktime` or affiliated repository (if contributed extension), inside `sktime.[name_of_task]`; rename the file and update the file docstring appropriately.
4. address the "todo" parts. Usually, this means: changing the name of the class, setting the tag values, specifying hyper-parameters, filling in `__init__`, `_fit`, `_predict` and/or other methods (for details see the extension template). You can add private methods as long as they do not override the default public interface. For more details, see the extension template.
5. to test your estimator manually: import your estimator and run it in the basic vignettes above.
6. to test your estimator automatically: call `sktime.tests.test_all_estimators.check_estimator` on your estimator. You can call this on a class or object instance. Ensure you have specified test parameters in the `get_test_params` method, according to the extension template.

In case of direct contribution to `sktime` or one of its affiliated packages, additionally:
* add yourself as an author to the code, and to the `CODEOWNERS` for the new estimator file(s).
* create a pull request that contains only the new estimators (and their inheritance tree, if it's not just one class), as well as the automated tests as described above.
* in the pull request, describe the estimator and optimally provide a publication or other technical reference for the strategy it implements.
* before making the pull request, ensure that you have all necessary permissions to contribute the code to a permissive license (BSD-3) open source project.

---

### Credits: notebook 5 - sktime engineering perspective

notebook creation: fkiraly

General credit to `sklearn` and `sktime` contributors